In [13]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Set up the scope for the APIs
scope = ["https://www.googleapis.com/auth/spreadsheets.readonly", 
         "https://www.googleapis.com/auth/drive.readonly"]

# Provide the path to the credentials JSON file you downloaded
creds = ServiceAccountCredentials.from_json_keyfile_name('/Users/alexandercappelen/Documents/keys/frb-elite-88e4dcc7ec5c.json', scope)

# Authorize and create the client
client = gspread.authorize(creds)

# Open the sheet by name
player_earned = client.open("frb-volley-game-stats").worksheet("player-offense")

# Fetch all records (rows) from the sheet
player_earned_data = player_earned.get_all_records()

In [14]:
# Print the data or convert it into a DataFrame
import pandas as pd
df = pd.DataFrame(player_earned_data)

df["error-pct"] = df["attack-errors"] / df["attack"]
df

,date,match-id,match,set,player,serves,aces,serve-error,points-won,point-scoring-pct,attack,kills,attack-errors,kill-pct,kill-effic,error-pct
0,08/12/2024,141108,frb-gentofte,1,Lasse,2,1,0,1,50.0,1,1,0,100.0,1.0,0.0
1,08/12/2024,141108,frb-gentofte,1,Nico,0,0,0,0,0.0,0,0,0,0.0,0.0,NaN
2,08/12/2024,141108,frb-gentofte,1,Kristian,5,0,2,3,60.0,2,1,0,50.0,0.5,0.0
3,08/12/2024,141108,frb-gentofte,1,Ando,0,0,0,0,0.0,0,0,0,0.0,0.0,NaN
4,08/12/2024,141108,frb-gentofte,1,Bosse,0,0,0,0,0.0,0,0,0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
743,29/03/2025,141144,hvidovre-frb,3,Mads,0,0,0,0,0.0,0,0,0,0.0,0.0,NaN
744,29/03/2025,141144,hvidovre-frb,3,Martin,2,0,2,0,0.0,10,10,0,100.0,1.0,0.0
745,29/03/2025,141144,hvidovre-frb,3,Mikkel,0,0,0,0,0.0,0,0,0,0.0,0.0,NaN
746,29/03/2025,141144,hvidovre-frb,3,Frederik,0,0,0,0,0.0,0,0,0,0.0,0.0,NaN


In [15]:
summary = df.groupby('player').agg(
    attack_attempts=('attack', 'sum'),
    total_kills=('kills', 'sum'),
    attack_errors=('attack-errors', 'sum')
).reset_index()

summary["kill_pct"] = ((summary["total_kills"]) / summary["attack_attempts"] * 100).round(0)
summary["error_pct"] = ((summary["attack_errors"]) / summary["attack_attempts"] * 100).round(0)
summary["kill_effic"] = ((summary["total_kills"] - summary["attack_errors"]) / summary["attack_attempts"])#.round(3)
summary = summary.dropna()
summary["kill_pct"] = summary["kill_pct"].astype(int)
summary["error_pct"] = summary["error_pct"].astype(int)
summary["kill_effic"] = summary["kill_effic"].round(3)
summary

,player,attack_attempts,total_kills,attack_errors,kill_pct,error_pct,kill_effic
1,Ando,14,8,1,57,7,0.500
2,Bo,37,14,7,38,19,0.189
3,Boerme,118,46,20,39,17,0.220
4,Bosse,27,9,5,33,19,0.148
5,Frederik,4,1,1,25,25,0.000
6,Gustav,73,27,12,37,16,0.205
7,Kristian,79,37,11,47,14,0.329
8,Lasse,56,22,7,39,12,0.268
9,Mads,21,3,0,14,0,0.143
10,Martin,247,109,45,44,18,0.259


In [16]:
import json

# Convert DataFrame to JSON
data = summary.to_dict(orient="records")  # Convert DataFrame rows to list of dictionaries
with open("../player-offense-summary.json", "w") as f:
    json.dump(data, f, indent=4)